# Dowload and clean german Wikipedia dump

Dieses Notebook ist das Hauptnotebook, welches den Wikipedia Dump herunterlädt, diesen aufsplittet in exzellente und nicht exzellente artikel und anschließend eine grundlegende Datenaufbereitung durchführt. Die Aufgaben 1 & 2 sind für eine bessere Übersicht in den folgenden seperaten Notebooks bearbeitet worden:

Aufgabe 1: Klassifizierung der Artikel 

Aufgabe 2: Keyword extraktion

### Install packages

In [1]:
! pip install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


### Import Packages

In [2]:
# imports

# regex
import re
import sys
import os
import bz2
import requests
import shutil

# package to read wikipedia dump
import mwxml
# packages for cleaning the data
import html2text
import wikitextparser as wtp

# packages for multithreading
from threading import Thread


### Static Variables / Config

In [3]:
# static var
DUMP_URL = 'https://dumps.wikimedia.org/dewiki/latest/dewiki-latest-pages-articles.xml.bz2'
DUMP_FILE_ZIP = './dewiki-latest-pages-articles.xml.bz2'
DUMP_FILE_ENTPACKT = './dewiki-latest-pages-articles.xml'

EXZELLENT_FOLDER = './data/exzellent'
NOT_EXZELLENT_FOLDER = './data/not_exzellent'
SUBSET_FOLDER = './data/subset'

### Funktionen definieren

In [4]:
def clean_save_article(page: object, revision: object, is_excellent: bool) -> None:
    text = revision.text

    # filter if article is only redirect and has no text 
    PATTERN_REDIRECT = r"(#REDIRECT|#redirect|#WEITERLEITUNG)"
    
    if re.search(PATTERN_REDIRECT, revision.text):
        # with open(os.path.join('./data/trash', str(page.id) + '.txt'), "x") as f:
        #     f.write(page.title + "\n" + text)
        return

    # entnommen aus: https://github.com/daveshap/PlainTextWikipedia
    # Plain Text
    text = wtp.parse(text).plain_text()  
    # Remove HTML
    text = html2text.html2text(text)
    # Replace newlines
    text = text.replace('\\n', ' ')
    # Replace excess whitespace
    text = re.sub('\s+', ' ', text)
    # end entnommen aus

    
    if is_excellent:
        # filter excellent label from article (just to be sure is not in article anymore - usually the html2text function filtes these tags)
        text = text.replace('\{\{Exzellent|', '\{\{')
        # set target folder based on label
        target_folder = EXZELLENT_FOLDER
    else: 
        # set target folder based on label
        target_folder = NOT_EXZELLENT_FOLDER
    
    # save in target folder and add Wikipedia title in first line of document
    with open(os.path.join(target_folder, str(page.id) + '.txt'), "x") as f:
        f.write(page.title + "\n" + text)

### Download XML Dump herunterladen und Chunkweise abspeichern

Herunterladen des Wikipedia Dumps mit allen deutschsprachigen Artikeln von wikimedia.

In [5]:
# Funktion zum Herunterladen der Datei
def download_file(url, file_path):
    response = requests.get(url, stream=True)
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

# Herunterladen des Wikipedia-Artikeldumps
download_file(DUMP_URL, DUMP_FILE_ZIP)

KeyboardInterrupt: 

### XML Dump entpacken

In [ ]:
# unzip the xml-dump and save it
with open(DUMP_FILE_ENTPACKT, 'wb') as new_file, bz2.BZ2File(DUMP_FILE_ZIP, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

### Artikel aufbereiten und sortieren nach Label

In [6]:
# pythonhosted.org
PATTERN = r"\{\{Exzellent\|"
dump = mwxml.Dump.from_file(open(DUMP_FILE_ENTPACKT))

excellent_count = 0
not_excellent_count = 0

print("### Wikipedia Dump ###")
print(dump.site_info.name, dump.site_info.dbname)

print("### Exzelente Aritkel ###")
if not os.path.isdir(EXZELLENT_FOLDER):
    os.makedirs(EXZELLENT_FOLDER)
    
# for schleifen entnommen aus: 
for idx_page, page in enumerate(dump):
    for idx_revision, revision in enumerate(page):
        if revision.text is not None:
            x = re.search(PATTERN, revision.text)
            if x is not None:
                # finde the excellent articles
                excellent_count += 1
                is_excellent = True
            else:
                not_excellent_count += 1
                is_excellent= False

            # start basic cleaning of article in seperated Thread for better performance
            Thread(target=clean_save_article, args=(page, revision, is_excellent)).start()

            # update output
            sys.stdout.write('\r Verarbeitete Artikel: ' + str(excellent_count + not_excellent_count) + ' davon exzellent: ' + str(excellent_count))
            sys.stdout.flush()

print('\n Anzahl der exzellenten Artikel: ', str(excellent_count))

### Wikipedia Dump ###
Wikipedia dewiki
### Exzelente Aritkel ###
 Verarbeitete Artikel: 5 davon exzellent: 0

 Verarbeitete Artikel: 6 davon exzellent: 0

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/01/1jhjxsz50y902wkbvgv3nj5r0000gn/T/ipykernel_18094/3831573612.py", line 34, in clean_save_article
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.

 Verarbeitete Artikel: 7 davon exzellent: 0

    self._target(*self._args, **self._kwargs)
  File "/var/folders/01/1jhjxsz50y902wkbvgv3nj5r0000gn/T/ipykernel_18094/3831573612.py", line 34, in clean_save_article
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
    return io_open(file, *args, **kwargs)
FileNotFoundError: [Errno 2] No such file or directory: './data/not_exzellent/7.txt'


...

  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/01/1jhjxsz50y902wkbvgv3nj5r0000gn/T/ipykernel_18094/3831573612.py", line 34, in clean_save_article
Exception in thread Thread-3146:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
    return io_open(file, *args, **kwargs)
FileNotFoundError: [Errno 

 Verarbeitete Artikel: 3158 davon exzellent: 119

    return io_open(file, *args, **kwargs)
FileNotFoundError: [Errno 2] No such file or directory: './data/not_exzellent/4714.txt'
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
Exception in thread Thread-3161:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-3160:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-3151:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    return io_open(file, *args, **kwargs)
FileNotFoundError: [Errno 2] No such file or dire

KeyboardInterrupt: 

Exception in thread Thread-3142:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/01/1jhjxsz50y902wkbvgv3nj5r0000gn/T/ipykernel_18094/3831573612.py", line 34, in clean_save_article
  File "/Users/sophiabuehl/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
Exception in thread Thread-3158:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
        self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/

### Subset generieren

In [20]:
# number overall articles
SUBSET_SIZE = 3000

articles_exzellent = os.listdir(EXZELLENT_FOLDER)
articles_not_exzellent = os.listdir(NOT_EXZELLENT_FOLDER)

number_exzellent:int = len(articles_exzellent)
number_not_exzellent:int = len(articles_not_exzellent)

ratio:float = float((number_exzellent / number_not_exzellent))

print("Gesamt Anzahl exzellenter Artikel: ", number_not_exzellent)
print("Gesamt Anzahl nicht exzellenter Artikel: ", number_exzellent)
print("Gesamtanzahl Artikel: ", (number_exzellent + number_not_exzellent))

for idx, article in enumerate(articles_exzellent):
    shutil.copyfile(os.path.join(EXZELLENT_FOLDER,article), os.path.join(SUBSET_FOLDER,'exzellent',article))
    if idx >= int(ratio * SUBSET_SIZE):
        print("Anzahl exzellente Artikel Subset: ", idx + 1)
        break

for idx, article in enumerate(articles_not_exzellent):
    shutil.copyfile(os.path.join(NOT_EXZELLENT_FOLDER,article), os.path.join(SUBSET_FOLDER,'not_exzellent',article))
    if idx >= int((1 - ratio) * SUBSET_SIZE):
        print("Anzahl nicht exzellente Artikel Subset: ", idx + 1)
        break

Gesamt Anzahl exzellenter Artikel:  2666629
Gesamt Anzahl nicht exzellenter Artikel:  2689
Gesamtanzahl Artikel:  2669318
Anzahl exzellente Artikel Subset:  4
Anzahl nicht exzellente Artikel Subset:  2997
